# AgentsVille Trip Planner - Projeto Local

In this project, you'll implement an AI system to help you plan a trip--to the wonderful city of AgentsVille!

Your AI system will demonstrate advanced LLM reasoning techniques including:

1. **Role-Based Prompting** - Your agent will act as a specialized travel planner
2. **Chain-of-Thought Reasoning** - Step-by-step planning of itineraries
3. **ReAct Prompting** - Thought → Action → Observation cycles
4. **Feedback Loops** - Self-evaluation using tools in the ReAct loop to find mistakes and improve plans

You'll simulate external API calls to gather weather data and activities. Then, process this information to create personalized travel itineraries based on interests and other constraints. Last, you'll implement a feedback loop to refine the plan.

Your task is to build a travel agent that can plan the perfect AgentsVille vacation!

## Initial Setup

Let's start with setting up our environment and defining the vacation details.

In [1]:
# Carregar variáveis de ambiente do arquivo .env
from dotenv import load_dotenv
import os

# Carrega as variáveis do arquivo .env
load_dotenv()

# Verifica se a API key foi carregada
if os.getenv("OPENAI_API_KEY"):
    print("✅ API Key da OpenAI carregada com sucesso!")
else:
    print("❌ Erro: API Key da OpenAI não encontrada. Verifique seu arquivo .env")

✅ API Key da OpenAI carregada com sucesso!


In [ ]:
# Install required packages if not already installed
# No changes needed here.
%pip install -q json-repair==0.47.1 numexpr==2.11.0 openai==1.74.0 pandas==2.3.0 pydantic==2.11.7 python-dotenv==1.1.0

In [2]:
# Configurar o cliente OpenAI usando sua API key local
from openai import OpenAI

# Cliente OpenAI usando a API key do seu .env
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Cliente OpenAI configurado com sucesso!")

✅ Cliente OpenAI configurado com sucesso!


In [3]:
# Definir os modelos disponíveis da OpenAI
from enum import Enum

class OpenAIModel(str, Enum):
    GPT_4O = "gpt-4o"  # Modelo mais poderoso
    GPT_4O_MINI = "gpt-4o-mini"  # Rápido e acessível (recomendado)
    GPT_4_TURBO = "gpt-4-turbo"  # Alternativa robusta
    GPT_35_TURBO = "gpt-3.5-turbo"  # Mais econômico

# Modelo padrão para este projeto (você pode mudar conforme necessário)
MODEL = OpenAIModel.GPT_4O_MINI

print(f"✅ Usando modelo: {MODEL}")

✅ Usando modelo: OpenAIModel.GPT_4O_MINI


## Define Vacation Details

Let's encode the details of our vacation in JSON format and verify it using Pydantic.

In [4]:
# The Vacation Info Data Structure
# No changes needed here, but you may choose to personalize the data.

VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Yuri",
            "age": 30,
            "interests": ["tennis", "cooking", "comedy", "technology"],
        },
        {
            "name": "Hiro",
            "age": 25,
            "interests": ["reading", "music", "theatre", "art"],
        },
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",
    "date_of_departure": "2025-06-12",
    "budget": 130,
}

In [5]:
# Validate the data structure using Pydantic
# TODO: Fill in the missing parts marked with **********

from project_lib import Interest
from typing import List
from pydantic import BaseModel
import datetime
from pprint import pprint

class Traveler(BaseModel):
    """A traveler with a name, age, and list of interests.
    
    Attributes:
        name (str): The name of the traveler.
        age (int): The age of the traveler.
        interests (List[Interest]): A list of interests of the traveler.
    """
    name: str
    age: int
    interests: List[Interest]

class VacationInfo(BaseModel):
    """Vacation information including travelers, destination, dates, and budget.
    Attributes:
        travelers (List[Traveler]): A list of travelers.
        destination (str): The vacation destination.
        date_of_arrival (datetime.date): The date of arrival.
        date_of_departure (datetime.date): The date of departure.
        budget (int): The budget for the vacation in fictional currency units.
    """
    # SOLUTION: Fill in the 5 missing fields
    travelers: List[Traveler]
    destination: str
    date_of_arrival: datetime.date
    date_of_departure: datetime.date
    budget: int


# Validate the VacationInfo data structure
vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)

# Display the vacation info as a dictionary
pprint(vacation_info.model_dump())

# Check that VacationInfo contains the expected data
assert "travelers" in vacation_info.model_dump().keys(), "VacationInfo should contain 'travelers' key"
assert "destination" in vacation_info.model_dump().keys(), "VacationInfo should contain 'destination' key"
assert "date_of_arrival" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_arrival' key"
assert "date_of_departure" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_departure' key"
assert "budget" in vacation_info.model_dump().keys(), "VacationInfo should contain 'budget' key"
assert isinstance(vacation_info.travelers, list), "Travelers should be a list"
assert all(isinstance(traveler, Traveler) for traveler in vacation_info.travelers), "All travelers should be instances of Traveler class"
assert isinstance(vacation_info.date_of_arrival, datetime.date), "date_of_arrival should be a date"
assert isinstance(vacation_info.date_of_departure, datetime.date), "date_of_departure should be a date"
assert isinstance(vacation_info.budget, int), "budget should be an integer"

# If all assertions pass, print a success message
print("✅ VacationInfo data structure is valid!")

{'budget': 130,
 'date_of_arrival': datetime.date(2025, 6, 10),
 'date_of_departure': datetime.date(2025, 6, 12),
 'destination': 'AgentsVille',
 'travelers': [{'age': 30,
                'interests': [tennis, cooking, comedy, technology],
                'name': 'Yuri'},
               {'age': 25,
                'interests': [reading, music, theatre, art],
                'name': 'Hiro'}]}
✅ VacationInfo data structure is valid!


## Review Weather and Activity Schedules

Now that we have the trip details, we can retrieve the weather and activity schedules for the dates of the trip.

In [6]:
# The `call_weather_api_mocked` mocks calling a weather API to get weather data
# TODO: Fill in the missing parts marked with **********

from project_lib import call_weather_api_mocked
import pandas as pd

pd.set_option("display.max_colwidth", None)  # Show full content in DataFrame cells

weather_for_dates = [
    call_weather_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
    for ts in pd.date_range(
        # SOLUTION: Fill in the missing start and end dates from vacation_info
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
]

weather_for_dates_df = pd.DataFrame(weather_for_dates)

weather_for_dates_df

,date,city,temperature,temperature_unit,condition,description
0,2025-06-10,AgentsVille,31,celsius,clear,A bright and sunny day in AgentsVille with clear skies and warm temperatures. Perfect weather for outdoor activities!
1,2025-06-11,AgentsVille,34,celsius,partly cloudy,"A warm day with periods of sunshine and mixed clouds, making it a perfect opportunity for outdoor activities."
2,2025-06-12,AgentsVille,28,celsius,thunderstorm,"A thunderstorm is expected to roll in during the afternoon, bringing heavy rain and gusty winds. The atmosphere will feel charged with humidity, creating a sultry and dramatic setting as clouds build in the sky."


In [7]:
# The `call_activities_api_mocked` function returns the activities for a given date and city.
# TODO: Fill in the missing parts marked with **********

from project_lib import call_activities_api_mocked

activities_for_dates = [
    activity
    for ts in pd.date_range(
        # SOLUTION: Fill in the missing start and end dates from vacation_info
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
    for activity in call_activities_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
]

activities_for_dates_df = pd.DataFrame(activities_for_dates)

activities_for_dates_df

,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00,2025-06-10 11:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[technology]
1,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00,2025-06-10 13:30,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[cooking, tennis]"
2,event-2025-06-10-2,Artful Athletics: Paint & Play Extravaganza,2025-06-10 15:00,2025-06-10 17:00,"Creative Courts Park, AgentsVille","Join us for an exciting afternoon at Creative Courts Park, where the worlds of art and sports collide! At 'Artful Athletics: Paint & Play Extravaganza', you'll participate in collaborative outdoor murals inspired by your favorite sports, and then get moving with fun, friendly sports mini-games. Whether you love painting or playing, this event celebrates creativity, teamwork, and the joy of movement under the open sky. Perfect for art lovers and sports enthusiasts alike—come ready to express yourself and get active! (Event is held outdoors; in case of rain, we move indoors to the Community Gym nearby.)",15,"[art, sports]"
3,event-2025-06-10-3,AgentsVille Twilight Writing Escape,2025-06-10 19:00,2025-06-10 21:00,"The Ink Loft, 12 Quill Lane, AgentsVille","Join fellow writers for an inspiring evening at The Ink Loft, where words flow as freely as the coffee! This writing-themed event welcomes all—novelists, poets, bloggers, or anyone with a passion for storytelling. Set indoors in AgentsVille's coziest lounge, enjoy writing games, group prompts, and opportunities to read your work aloud. Connect, create, and celebrate the art of writing in this creative indoor haven.",15,"[writing, reading, art]"
4,event-2025-06-11-0,Morning Groove Dance Party,2025-06-11 09:00,2025-06-11 10:30,"Rhythm Hall, Center Plaza, AgentsVille","Start your day with energy and joy at the Morning Groove Dance Party! This lively event welcomes dancers of all levels to join a vibrant indoor session filled with upbeat music and fun routines. Whether you love modern pop, Latin beats, or classic disco, our dance instructors will guide you to move and groove. Connect with fellow dance lovers in the colorful atmosphere of Rhythm Hall. Perfect for fans of dancing, music, and fitness. Let the rhythm move you! (Indoor event.)",15,"[dancing, music, fitness]"
5,event-2025-06-11-1,Tech Lunch & Learn: AI Frontiers,2025-06-11 12:00,2025-06-11 13:30,"The Digital Atrium, AgentsVille","Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect with others passionate about technology, AI, and the digital world. Whether you're a seasoned developer or just curious about tech, this event is for you! Related interests: technology, music (sound tech demos), photography (AI imaging), writing (AI cr

## The ItineraryAgent

First we will review the Pydantic objects used for defining the output of our agent.

In [8]:
# Review the data structure we will use for representing a TravelPlan
# No changes are needed here.

class Weather(BaseModel):
    temperature: float
    temperature_unit: str
    condition: str


class Activity(BaseModel):
    activity_id: str
    name: str
    start_time: datetime.datetime
    end_time: datetime.datetime
    location: str
    description: str
    price: int
    related_interests: List[Interest]


class ActivityRecommendation(BaseModel):
    activity: Activity
    reasons_for_recommendation: List[str]


class ItineraryDay(BaseModel):
    date: datetime.date
    weather: Weather
    activity_recommendations: List[ActivityRecommendation]


class TravelPlan(BaseModel):
    city: str
    start_date: datetime.date
    end_date: datetime.date
    total_cost: int
    itinerary_days: List[ItineraryDay]

In [9]:
# Specify the Chain-of-Thought (CoT) prompt for the Itinerary Agent.
# TODO: Fill in the missing parts marked with **********

import json 
from project_lib import ChatAgent
from typing import Optional

# SOLUTION: Complete prompt with Role + Task + Output Format + Context
ITINERARY_AGENT_SYSTEM_PROMPT = f"""
# Role
You are an expert AI Travel Planning Agent for AgentsVille, specializing in creating personalized, detailed travel itineraries. You have deep knowledge of weather considerations, budget management, and activity recommendations.

## Task

Generate a comprehensive day-by-day travel itinerary based on the provided traveler preferences and constraints. Follow this step-by-step planning process:

### Step 1: Analyze Traveler Preferences
- Review each traveler's interests carefully
- Identify activities from the available data that match their interests
- Ensure EVERY traveler has at least one activity matching their interests

### Step 2: Review Weather Conditions
- Examine the weather forecast for each day of the trip
- Identify days with inclement weather (rain, thunderstorms)
- CRITICAL: Outdoor-only activities MUST be avoided during rain/thunderstorms
- For bad weather days, prioritize indoor activities or activities with indoor backup options mentioned in their descriptions

### Step 3: Budget Planning and Cost Calculation
- Calculate the total cost by summing ALL activity prices
- CRITICAL: Total cost MUST NOT exceed the budget
- Double-check your arithmetic: sum each activity price carefully
- Consider using mental calculation or stating: "Activity A costs X + Activity B costs Y = Total Z"

### Step 4: Daily Activity Selection
- Select at least ONE activity for EACH day (this is mandatory)
- Choose activities at appropriate times (morning, lunch, afternoon, evening)
- Ensure selected activities DO NOT overlap in time
- CRITICAL: Use ONLY activity_ids that exist in the provided activities data
- Do NOT invent or hallucinate activities

### Step 5: Quality Assurance Check
- Verify all activity_ids exist in the available data
- Confirm dates match the arrival and departure dates
- Re-verify total cost calculation is accurate
- Ensure no outdoor-only activities are scheduled during inclement weather

## Output Format

You MUST respond in two sections:

ANALYSIS:
* Step 1 - Traveler Interests: [List each traveler and analyze what activities match their interests]
* Step 2 - Weather Review: [Review weather for each day, note any rain/thunderstorms, identify which days need indoor activities]
* Step 3 - Budget Calculation: [Show calculation: Activity1_price + Activity2_price + ... = Total. Verify total <= budget]
* Step 4 - Daily Activities: [For each day, explain which activities you selected and why]
* Step 5 - Final Verification: [Confirm all requirements met: dates correct, budget OK, interests satisfied, weather appropriate]

FINAL OUTPUT:

```json
{json.dumps(TravelPlan.model_json_schema(), indent=2)}
```

## Context

### Vacation Information (User Input)
The user will provide this in their message.

### Available Weather Data
{json.dumps(weather_for_dates, indent=2)}

### Available Activities Data (ONLY use activities from this list)
{json.dumps(activities_for_dates, indent=2)}

## Critical Constraints (MUST follow these)
1. Outdoor-only activities MUST be avoided during rain, thunderstorms
2. Total cost MUST NOT exceed budget
3. Each day MUST have at least ONE activity
4. ALL activity_ids MUST exist in the available activities data (no hallucinations)
5. Activities MUST match travelers' interests when possible
6. Start and end dates MUST match exactly
7. City MUST be correct
"""

assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'TASK' section"
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'OUTPUT FORMAT' section"


class ItineraryAgent(ChatAgent):
    """An agent that plans itineraries based on vacation information, weather, and activities."""
    system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT

    def get_itinerary(self, vacation_info: VacationInfo, model: Optional[OpenAIModel] = None) -> TravelPlan:
        """Generates a travel itinerary based on the provided vacation information."""
        from project_lib import print_in_box
        response = (self.chat(
            user_message=vacation_info.model_dump_json(indent=2),
            add_to_messages=False,
            model=model or self.model,
        ) or "").strip()

        print_in_box(response, "Raw Response")

        # Parse the response
        json_text = response.strip()

        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()

        try:
            travel_plan = TravelPlan.model_validate_json(json_text)
            return travel_plan
        except Exception as e:
            print("Error validating the following text as TravelPlan JSON:")
            print(json_text)
            raise

itinerary_agent = ItineraryAgent(client=client, model=MODEL)


╔══════════════════════════════════════════[ ItineraryAgent - System Prompt ]══════════════════════════════════════════╗
║ You are a helpful assistant.                                                                                         ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝


# Specify the Chain-of-Thought (CoT) prompt for the Itinerary Agent.
# TODO: Fill in the missing parts marked with **********

import json 
from project_lib import ChatAgent
from typing import Optional

# SOLUÇÃO: Prompt completo com Role + Task + Output Format + Context
ITINERARY_AGENT_SYSTEM_PROMPT = """
# Role
You are an expert AI Travel Planning Agent for AgentsVille, specializing in creating personalized, detailed travel itineraries. You have deep knowledge of weather considerations, budget management, and activity recommendations.

## Task

Generate a comprehensive day-by-day travel itinerary based on the provided traveler preferences and constraints. Follow this step-by-step planning process:

### Step 1: Analyze Traveler Preferences
- Review each traveler's interests carefully
- Identify activities from the available data that match their interests
- Ensure EVERY traveler has at least one activity matching their interests

### Step 2: Review Weather Conditions
- Examine the weather forecast for each day of the trip
- Identify days with inclement weather (rain, thunderstorms)
- CRITICAL: Outdoor-only activities MUST be avoided during rain/thunderstorms
- For bad weather days, prioritize indoor activities or activities with indoor backup options mentioned in their descriptions

### Step 3: Budget Planning and Cost Calculation
- Calculate the total cost by summing ALL activity prices
- CRITICAL: Total cost MUST NOT exceed the budget
- Double-check your arithmetic: sum each activity price carefully
- Consider using mental calculation or stating: "Activity A costs X + Activity B costs Y = Total Z"

### Step 4: Daily Activity Selection
- Select at least ONE activity for EACH day (this is mandatory)
- Choose activities at appropriate times (morning, lunch, afternoon, evening)
- Ensure selected activities DO NOT overlap in time
- CRITICAL: Use ONLY activity_ids that exist in the provided activities data
- Do NOT invent or hallucinate activities

### Step 5: Quality Assurance Check
- Verify all activity_ids exist in the available data
- Confirm dates match the arrival and departure dates
- Re-verify total cost calculation is accurate
- Ensure no outdoor-only activities are scheduled during inclement weather

## Output Format

You MUST respond in two sections:

ANALYSIS:
* Step 1 - Traveler Interests: [List each traveler and analyze what activities match their interests]
* Step 2 - Weather Review: [Review weather for each day, note any rain/thunderstorms, identify which days need indoor activities]
* Step 3 - Budget Calculation: [Show calculation: Activity1_price + Activity2_price + ... = Total. Verify total <= budget]
* Step 4 - Daily Activities: [For each day, explain which activities you selected and why]
* Step 5 - Final Verification: [Confirm all requirements met: dates correct, budget OK, interests satisfied, weather appropriate]

FINAL OUTPUT:

```json
""" + json.dumps(TravelPlan.model_json_schema(), indent=2) + """
```

## Context

### Vacation Information (User Input)
The user will provide this in their message.

### Available Weather Data
""" + json.dumps(weather_for_dates, indent=2) + """

### Available Activities Data (ONLY use activities from this list)
""" + json.dumps(activities_for_dates, indent=2) + """

## Critical Constraints (MUST follow these)
1. Outdoor-only activities MUST be avoided during rain, thunderstorms
2. Total cost MUST NOT exceed budget
3. Each day MUST have at least ONE activity
4. ALL activity_ids MUST exist in the available activities data (no hallucinations)
5. Activities MUST match travelers' interests when possible
6. Start and end dates MUST match exactly
7. City MUST be correct
"""

assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'TASK' section"
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'OUTPUT FORMAT' section"


class ItineraryAgent(ChatAgent):
    """An agent that plans itineraries based on vacation information, weather, and activities."""
    system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT

    def get_itinerary(self, vacation_info: VacationInfo, model: Optional[OpenAIModel] = None) -> TravelPlan:
        """Generates a travel itinerary based on the provided vacation information."""
        from project_lib import print_in_box
        response = (self.chat(
            user_message=vacation_info.model_dump_json(indent=2),
            add_to_messages=False,
            model=model or self.model,
        ) or "").strip()

        print_in_box(response, "Raw Response")

        # Parse the response
        json_text = response.strip()

        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()

        try:
            travel_plan = TravelPlan.model_validate_json(json_text)
            return travel_plan
        except Exception as e:
            print("Error validating the following text as TravelPlan JSON:")
            print(json_text)
            raise

itinerary_agent = ItineraryAgent(client=client, model=MODEL)

In [10]:
# Generate the travel itinerary
# VERSÃO COM DEBUG PARA IDENTIFICAR PROBLEMAS

print("=" * 80)
print("INICIANDO GERAÇÃO DO ITINERÁRIO")
print("=" * 80)

# Verificação 1: Agent existe?
try:
    print(f"✅ Agent encontrado: {itinerary_agent.name}")
    print(f"✅ Model configurado: {itinerary_agent.model}")
except NameError:
    print("❌ ERRO: itinerary_agent não foi criado!")
    print("   Solução: Execute a célula 10 novamente")
    raise

# Verificação 2: VacationInfo existe?
try:
    print(f"✅ VacationInfo encontrado: {vacation_info.destination}")
    print(f"   Travelers: {len(vacation_info.travelers)}")
    print(f"   Budget: {vacation_info.budget}")
except NameError:
    print("❌ ERRO: vacation_info não foi criado!")
    print("   Solução: Execute a célula 6 novamente")
    raise

# Verificação 3: TravelPlan definido?
try:
    print(f"✅ TravelPlan class definida: {TravelPlan.__name__}")
except NameError:
    print("❌ ERRO: TravelPlan não foi definida!")
    print("   Solução: Execute a célula 9 novamente")
    raise

print("\n" + "=" * 80)
print("CHAMANDO O LLM PARA GERAR ITINERÁRIO...")
print("=" * 80)
print("⏳ Isso pode levar 30-60 segundos. Aguarde...")
print()

# Tentar gerar o itinerário com tratamento de erros
try:
    travel_plan_1 = itinerary_agent.get_itinerary(
        vacation_info=vacation_info,
        model=MODEL,
    )
    
    if travel_plan_1 is not None:
        print("\n" + "=" * 80)
        print("✅ SUCESSO! Itinerário gerado com sucesso!")
        print("=" * 80)
        print(f"Cidade: {travel_plan_1.city}")
        print(f"Período: {travel_plan_1.start_date} até {travel_plan_1.end_date}")
        print(f"Total de dias: {len(travel_plan_1.itinerary_days)}")
        print(f"Custo total: {travel_plan_1.total_cost}")
        print()
        
        # Mostrar resumo de cada dia
        for day in travel_plan_1.itinerary_days:
            print(f"  📅 {day.date}: {len(day.activity_recommendations)} atividade(s)")
        
        print("\n✅ Variável 'travel_plan_1' criada com sucesso!")
        print("✅ Pode prosseguir para a célula 13")
    else:
        print("\n❌ ERRO: travel_plan_1 é None!")
        print("   O LLM retornou None em vez de um TravelPlan")
        raise ValueError("travel_plan_1 is None")
        
except Exception as e:
    print("\n" + "=" * 80)
    print("❌ ERRO DURANTE A GERAÇÃO DO ITINERÁRIO")
    print("=" * 80)
    print(f"Tipo do erro: {type(e).__name__}")
    print(f"Mensagem: {str(e)}")
    print()
    print("🔍 POSSÍVEIS CAUSAS:")
    print("1. Resposta do LLM não está em formato JSON válido")
    print("2. Timeout na API OpenAI")
    print("3. Problema com a API key")
    print("4. Prompt muito longo (excedeu limite de tokens)")
    print()
    print("🔧 SOLUÇÕES:")
    print("1. Tente executar novamente (pode ser timeout temporário)")
    print("2. Verifique sua API key no .env")
    print("3. Tente com um model menor: MODEL = OpenAIModel.GPT_41_NANO")
    print("4. Reduza o período da viagem (menos dias)")
    print()
    
    # Re-raise para mostrar o traceback completo
    raise

print("\n" + "=" * 80)
print("FIM DA EXECUÇÃO DA CÉLULA 11")
print("=" * 80)

INICIANDO GERAÇÃO DO ITINERÁRIO
✅ Agent encontrado: ItineraryAgent
✅ Model configurado: OpenAIModel.GPT_4O_MINI
✅ VacationInfo encontrado: AgentsVille
   Travelers: 2
   Budget: 130
✅ TravelPlan class definida: TravelPlan

CHAMANDO O LLM PARA GERAR ITINERÁRIO...
⏳ Isso pode levar 30-60 segundos. Aguarde...


╔═══════════════════════════════════════════[ ItineraryAgent - User Prompt ]═══════════════════════════════════════════╗
║ {                                                                                                                    ║
║   "travelers": [                                                                                                     ║
║     {                                                                                                                ║
║       "name": "Yuri",                                                                                                ║
║       "age": 30,                                                                   

ValidationError: 1 validation error for TravelPlan
  Invalid JSON: expected ident at line 1 column 2 [type=json_invalid, input_value="It looks like you're pla...ur trip to AgentsVille!", input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

## Evaluating the Itinerary

We've successfully created an itinerary, but how do we know if it's any good?

In [ ]:
# Helper functions for running the evaluation functions
# No change needed here.

class AgentError(Exception):
    pass


class EvaluationResults(BaseModel):
    success: bool
    failures: List[str]
    eval_functions: List[str]


def get_eval_results(vacation_info, final_output, eval_functions) -> EvaluationResults:
    """Evaluates the final output against evaluation functions."""
    from project_lib import print_in_box
    if not isinstance(vacation_info, VacationInfo):
        raise ValueError("vacation_info must be an instance of VacationInfo")
    if not isinstance(final_output, TravelPlan):
        raise ValueError("final_output must be an instance of TravelPlan")
    if not isinstance(eval_functions, list) or not all(
        callable(fn) for fn in eval_functions
    ):
        raise ValueError("eval_functions must be a list of callable functions")
    eval_results = []
    for eval_fn in eval_functions:
        try:
            eval_fn(vacation_info, final_output)
        except AgentError as e:
            error_msg = str(e)
            print_in_box(error_msg, title="Evaluation Error")
            print("\n\n")

            eval_results.append(error_msg)
    return EvaluationResults(
        success=len(eval_results) == 0,
        failures=eval_results,
        eval_functions=[fn.__name__ for fn in eval_functions],
    )

In [ ]:
# Basic evaluation functions
# No changes needed here.

def eval_start_end_dates_match(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that dates match."""
    if (
        vacation_info.date_of_arrival != final_output.start_date
        or vacation_info.date_of_departure != final_output.end_date
    ):
        raise AgentError(
            f"Dates do not match: {vacation_info.date_of_arrival} != {final_output.start_date} or {vacation_info.date_of_departure} != {final_output.end_date}"
        )

    if final_output.start_date > final_output.end_date:
        raise AgentError(
            f"Start date is after end date: {final_output.start_date} > {final_output.end_date}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_start_end_dates_match],
)

In [ ]:
# Evaluation functions related to the budget and total cost
# No changes needed here.


def eval_total_cost_is_accurate(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies total cost accuracy."""
    actual_total_cost = 0

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            actual_total_cost += activity_recommendation.activity.price

    stated_total_cost = int(final_output.total_cost)

    if actual_total_cost != stated_total_cost:
        raise AgentError(
            f"Stated total cost does not match calculated total cost: {actual_total_cost} != {stated_total_cost}"
        )
    
def eval_total_cost_is_within_budget(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies total cost is within budget."""
    stated_total_cost = int(final_output.total_cost)
    if stated_total_cost > vacation_info.budget:
        raise AgentError(
            f"Total cost exceeds budget: {stated_total_cost} > {vacation_info.budget}"
        )

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_total_cost_is_accurate, eval_total_cost_is_within_budget],
)

In [ ]:
# The final output contains copies of the activities, so we need to verify the copies are accurate
# No changes needed here.

def eval_itinerary_events_match_actual_events(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies events match actual events."""
    from project_lib import call_activity_by_id_api_mocked
    event_ids_not_matching = []
    event_ids_missing = []

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            event_id = activity_recommendation.activity.activity_id

            reference_event = call_activity_by_id_api_mocked(event_id)

            if reference_event is None:
                event_ids_missing.append(event_id)

            elif Activity(**reference_event) != activity_recommendation.activity:
                print(
                    "---\n"
                    f"Event ID {event_id} does not match the reference event:\n"
                    f"Reference Event: {reference_event}\n"
                    f"Activity Event: {activity_recommendation.activity.model_dump()}"
                )
                event_ids_not_matching.append(event_id)
            else:
                pass

    if event_ids_missing or event_ids_not_matching:
        raise AgentError(
            f"Event IDs missing: {event_ids_missing}\nEvent IDs not matching: {event_ids_not_matching}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_events_match_actual_events],
)

In [ ]:
# Check that the itinerary includes at least one activity matching each traveler's interests.
# No changes needed here.

def eval_itinerary_satisfies_interests(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies itinerary satisfies interests."""
    traveler_to_interests = {}
    traveler_to_interest_hit_counts = {}

    for traveler in vacation_info.travelers:
        traveler_to_interests[traveler.name] = traveler.interests
        traveler_to_interest_hit_counts[traveler.name] = 0

    for traveler_name, interests in traveler_to_interests.items():
        for itinerary_day in final_output.itinerary_days:
            for activity_recommendation in itinerary_day.activity_recommendations:
                matching_interests = set(traveler_to_interests[traveler_name]) & set(
                    activity_recommendation.activity.related_interests
                )

                if matching_interests:
                    traveler_to_interest_hit_counts[traveler_name] += 1
                    print(
                        f"✅ Traveler {traveler_name} has a match with interest {matching_interests} at {activity_recommendation.activity.name}"
                    )

    travelers_with_no_interest_hits = [
        traveler
        for traveler, interest_hit_count in traveler_to_interest_hit_counts.items()
        if interest_hit_count == 0
    ]

    if travelers_with_no_interest_hits:
        raise AgentError(
            f"Travelers {travelers_with_no_interest_hits} has no matches with the itinerary."
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_satisfies_interests],
)

In [ ]:
# Use an LLM to determine whether an event should be avoided due to weather conditions.
# TODO: Fill in the missing parts marked with **********

# SOLUTION: Complete prompt with Role + Task + Output + Examples
ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = """
# Role
You are an expert Weather and Activity Compatibility Analyst. Your job is to determine if an activity should be avoided due to weather conditions.

## Task
Evaluate whether the given activity is compatible with the given weather condition. 

Consider these factors:
1. If the activity is described as "outdoors" or "outdoor-only" and the weather is inclement (rain, thunderstorm), it is INCOMPATIBLE
2. If the activity description mentions it is "indoors" or has "indoor backup options", it is COMPATIBLE regardless of weather
3. If there is not enough information about whether the activity is indoors or outdoors, assume it IS_COMPATIBLE (benefit of the doubt)
4. Indoor activities are always COMPATIBLE with any weather
5. Outdoor activities with backup plans mentioned in the description are COMPATIBLE even in bad weather

## Output Format

You must respond in exactly this format:

REASONING:
[Provide step-by-step reasoning:
1. Analyze if the activity is indoors or outdoors based on description
2. Check if the description mentions backup options or indoor alternatives
3. Evaluate if the weather condition is problematic for outdoor activities
4. Make final compatibility decision]

FINAL ANSWER:
[Write either "IS_COMPATIBLE" or "IS_INCOMPATIBLE" - use these exact words]

## Examples

Example 1:
Activity: Mountain Hiking Adventure
Description: Join us for an all-day outdoor hike through mountain trails. This event is entirely outdoors with no shelter available.
Weather Condition: thunderstorm

REASONING:
1. The activity is explicitly "entirely outdoors" with no indoor component
2. The description mentions "no shelter available", so no backup plan exists
3. The weather is "thunderstorm" which is dangerous for outdoor hiking
4. This combination is unsafe and the activity should be avoided

FINAL ANSWER:
IS_INCOMPATIBLE

Example 2:
Activity: Indoor Art Gallery Workshop
Description: Explore your creativity in our climate-controlled indoor art studio. All materials provided in a comfortable indoor setting.
Weather Condition: rainy

REASONING:
1. The activity is clearly "indoors" in an "art studio"
2. Description emphasizes "indoor setting" and "climate-controlled"
3. Weather is rainy, but this has no impact on indoor activities
4. Indoor activities are unaffected by outdoor weather conditions

FINAL ANSWER:
IS_COMPATIBLE

Example 3:
Activity: Summer Music Festival
Description: Enjoy live music at the outdoor park amphitheater. In case of rain, the event will move indoors to the adjacent Harmony Hall.
Weather Condition: thunderstorm

REASONING:
1. The activity starts as outdoor at an amphitheater
2. The description EXPLICITLY states: "In case of rain, the event will move indoors to the adjacent Harmony Hall"
3. Weather is thunderstorm, which would normally be problematic
4. However, the backup indoor venue makes this activity viable even in bad weather

FINAL ANSWER:
IS_COMPATIBLE

Example 4:
Activity: Botanical Garden Walking Tour
Description: Explore beautiful outdoor gardens and learn about local plants in this scenic outdoor tour.
Weather Condition: sunny

REASONING:
1. The activity is outdoors in botanical gardens
2. The weather is sunny, which is perfect for outdoor activities
3. No weather-related concerns exist
4. This is an ideal combination

FINAL ANSWER:
IS_COMPATIBLE

Example 5:
Activity: Technology Conference
Description: A day of tech talks and networking. Venue details to be announced.
Weather Condition: rainy

REASONING:
1. The activity description does not specify if it's indoors or outdoors
2. Technology conferences are typically held indoors, but this isn't explicitly stated
3. Following the principle of "benefit of the doubt" when information is unclear
4. Assume the activity can proceed normally

FINAL ANSWER:
IS_COMPATIBLE
""".strip()


def eval_activities_and_weather_are_compatible(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies no outdoor-only activities during inclement weather."""
    from project_lib import do_chat_completion

    activities_that_are_incompatible = []

    for itinerary_day in final_output.itinerary_days:
        weather_condition = itinerary_day.weather.condition

        for activity_recommendation in itinerary_day.activity_recommendations:
            resp = do_chat_completion(
                messages=[
                    {
                        "role": "system",
                        "content": ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT,
                    },
                    {
                        "role": "user",
                        "content": f"Activity: {activity_recommendation.activity.name}\nDescription: {activity_recommendation.activity.description}\nWeather Condition: {weather_condition}",
                    },
                ],
                client=client,
                model=OpenAIModel.GPT_41_NANO,
            )
    


            if "IS_COMPATIBLE" in (resp or ""):
                is_compatible = True
            elif "IS_INCOMPATIBLE" in (resp or ""):
                is_compatible = False
            else:
                raise RuntimeError(
                    f"Unexpected response from the model: {resp}. Expected 'IS_COMPATIBLE' or 'IS_INCOMPATIBLE'."
                )

            if is_compatible:
                print(
                    f"✅ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are compatible."
                )

            else:
                activities_that_are_incompatible.append(
                    activity_recommendation.activity.name
                )
                print(
                    f"❌ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are incompatible."
                )

    if activities_that_are_incompatible:
        raise AgentError(
            f"Activities that may be ruined by inclement weather: {activities_that_are_incompatible}"
        )


eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[
        eval_activities_and_weather_are_compatible
    ],
)

eval_results

In [ ]:
# Run all of the evaluation functions again
# No changes needed here.

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
]

eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

eval_results.model_dump()

## Defining the Tools

Our ItineraryRevisionAgent will be a ReAct-based agent that will use tools.

In [ ]:
# Helper function to generate tool descriptions from function docstrings
# No changes needed here.

def get_tool_descriptions_string(fns):
    """Generates a tool description from a function's docstring."""
    resp = ""
    for fn in fns:
        function_name = fn.__name__
        function_doc = fn.__doc__ or "No description provided."

        resp += f"* `{function_name}`: {function_doc}\n"

    return resp

In [ ]:
# Define the calculator tool that evaluates mathematical expressions.
# No changes needed here.

def calculator_tool(input_expression) -> float:
    """Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    """
    import numexpr as ne
    return float(ne.evaluate(input_expression))


assert calculator_tool("1 + 1") == 2.0

print(get_tool_descriptions_string([calculator_tool]))

In [ ]:
# Tool to fetch activities for a given date and city.
# TODO: Fill in the missing parts marked with **********

def get_activities_by_date_tool(date: str, city: str) -> List[dict]:
    """Retrieves all available activities for a specific date and city from the activities API.
    
    This tool is useful when you need to:
    - Find alternative activities for a specific date
    - Check what activities are available on a given day
    - Verify activity details before adding them to an itinerary
    - Replace activities that don't meet requirements
    
    Args:
        date (str): The date to search for activities in ISO format 'YYYY-MM-DD' (e.g., '2025-06-10')
        city (str): The city name to search in (e.g., 'AgentsVille')
    
    Returns:
        List[dict]: A list of activity dictionaries, where each dictionary contains:
            - activity_id (str): Unique identifier for the activity
            - name (str): Name of the activity
            - start_time (str): Start datetime in ISO format
            - end_time (str): End datetime in ISO format
            - location (str): Where the activity takes place
            - description (str): Detailed description of the activity
            - price (int): Cost in fictional currency units
            - related_interests (List[str]): List of interests this activity matches
    
    Example:
        activities = get_activities_by_date_tool(date="2025-06-10", city="AgentsVille")
        # Returns list of all activities available on 2025-06-10 in AgentsVille
    """
    from project_lib import call_activities_api_mocked
    resp = call_activities_api_mocked(date=date, city=city)

    return [Activity.model_validate(activity).model_dump() for activity in resp]



assert len(get_activities_by_date_tool("2025-06-10", "AgentsVille")) > 0

print(get_tool_descriptions_string([get_activities_by_date_tool]))

In [ ]:
# Tool to run all evaluation functions on a travel plan.
# No changes needed here.

def run_evals_tool(travel_plan: TravelPlan) -> dict:
    """Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    """
    if isinstance(travel_plan, dict):
        travel_plan = TravelPlan.model_validate(travel_plan)

    resp = get_eval_results(
        vacation_info=vacation_info,
        final_output=travel_plan,
        eval_functions=ALL_EVAL_FUNCTIONS,
    )
    return {
        # Show the success status and any failures
        "success": resp.success,
        "failures": resp.failures,
    }


print(get_tool_descriptions_string([run_evals_tool]))

In [ ]:
# Let's double check that the tool works as expected.
# You should see the same results as before
run_evals_tool(travel_plan=travel_plan_1)

In [ ]:
# A tool to return the final travel plan
# No changes needed here.

def final_answer_tool(final_output: TravelPlan) -> TravelPlan:
    """Returns the final travel plan

    Args:
        final_output (TravelPlan): The final travel plan to return.

    Returns:
        TravelPlan: The final travel plan.
    """
    return final_output


print(get_tool_descriptions_string([final_answer_tool]))

In [ ]:
# List of all tools available for the agent
# No changes needed here.

ALL_TOOLS = [
    calculator_tool,
    get_activities_by_date_tool,
    run_evals_tool,
    final_answer_tool,
]
print(get_tool_descriptions_string(ALL_TOOLS))

## The ItineraryRevisionAgent

The ItineraryRevisionAgent will refine the original itinerary iteratively using a ReAct-based approach.

In [ ]:
# Get the traveler's feedback and create a new evaluation function.
# No changes needed here.

TRAVELER_FEEDBACK = "I want to have at least two activities per day."


def eval_traveler_feedback_is_incorporated(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Checks if the traveler's feedback was incorporated into the revised travel plan."""

    agent = ChatAgent(
        system_prompt="""You are an expert in evaluating whether a travel plan incorporates traveler feedback.

    ## Output Format

    Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

        ANALYSIS:
        * [step-by-step analysis]


        FINAL OUTPUT:
        [FULLY_INCORPORATED, PARTIALLY_INCORPORATED, NOT_INCORPORATED, or UNKNOWN]
        REASON: [reasoning for the final output]

    """,
        client=client,
        model=OpenAIModel.GPT_41,
    )

    resp = agent.chat(
        f"""Traveler Feedback: {TRAVELER_FEEDBACK}
    Revised Travel Plan: {final_output.model_dump_json()}
    """,
    )
    if "FINAL OUTPUT:" not in resp:
        raise RuntimeError(
            f"Unexpected response from the model: {resp}. Expected 'FINAL OUTPUT:'."
        )
    if "FULLY_INCORPORATED" not in resp:
        final_output = resp.split("FINAL OUTPUT:")[-1].strip()
        raise AgentError(
            f"Traveler feedback was not successfully incorporated into the revised travel plan. Response: {final_output}"
        )

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
    eval_traveler_feedback_is_incorporated,
]

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

In [ ]:
# Define the ReAct system prompt for the Itinerary Revision Agent.
# TODO: Fill in the missing parts marked with **********

from project_lib import print_in_box

# SOLUTION: Complete ReAct prompt
ITINERARY_REVISION_AGENT_SYSTEM_PROMPT = f"""
# Role
You are an expert Itinerary Revision Agent that uses the ReAct (Reasoning and Acting) framework to systematically improve travel plans. You iteratively refine itineraries by reasoning about problems, taking actions with tools, and observing results.

## Task

Your task is to revise the provided travel itinerary to:
1. **Run initial evaluation**: ALWAYS start by calling run_evals_tool to understand current issues
2. **Incorporate traveler feedback**: The feedback is: "{TRAVELER_FEEDBACK}"
3. **Fix all identified issues**: Address each failure reported by evaluations
4. **Verify before completion**: Run run_evals_tool again BEFORE calling final_answer_tool to ensure success

## Available Tools

You have access to these tools:

{get_tool_descriptions_string(ALL_TOOLS)}

## ReAct Cycle: THINK-ACT-OBSERVE

You MUST follow this cycle iteratively:

### THOUGHT
In the THOUGHT section, you should:
- Analyze the current situation and what you've learned from previous observations
- Identify what information you need or what problem to solve next
- Explain your reasoning for choosing the next action
- Plan your approach step-by-step

### ACTION
In the ACTION section, you MUST call exactly ONE tool using this exact JSON format:

{{"tool_name": "<tool_name_here>", "arguments": {{"arg1": "value1", "arg2": "value2"}}}}

CRITICAL: Use the exact format above. Replace <tool_name_here> with actual tool name and provide appropriate arguments.

### OBSERVATION
After your ACTION, I will execute the tool and provide an OBSERVATION with the results. Use this information in your next THOUGHT.

## Workflow Requirements

You MUST follow this workflow:

1. **First Action**: ALWAYS call run_evals_tool first to see what needs fixing
2. **Fix Issues**: Based on eval failures, use appropriate tools:
   - Use calculator_tool if cost calculations are wrong
   - Use get_activities_by_date_tool if you need to find better activities
   - Modify the itinerary to address each failure
3. **Verify Changes**: Before final answer, call run_evals_tool again
4. **Only Exit When Success**: Call final_answer_tool ONLY when run_evals_tool shows success: true

## Example ReAct Cycle

THOUGHT:
I need to start by understanding what's wrong with the current itinerary. I'll run the evaluation tool to get a comprehensive assessment.

ACTION:
{{"tool_name": "run_evals_tool", "arguments": {{"travel_plan": <current_plan>}}}}

[I provide OBSERVATION with evaluation results showing failures]

THOUGHT:
The evaluation shows the total cost is calculated incorrectly. The stated total is 90 but the actual sum of activities is 95. I need to use the calculator to get the correct total.

ACTION:
{{"tool_name": "calculator_tool", "arguments": {{"input_expression": "20 + 15 + 18 + 25 + 17"}}}}

[I provide OBSERVATION: Tool calculator_tool called successfully with response: 95.0]

THOUGHT:
Good, the correct total is 95. Now I need to update the itinerary with the correct total cost and also add more activities to satisfy the traveler feedback...

[Continue until all issues resolved...]

THOUGHT:
I've addressed all issues. Now I must verify by running the evaluations one final time before providing the final answer.

ACTION:
{{"tool_name": "run_evals_tool", "arguments": {{"travel_plan": <revised_plan>}}}}

[I provide OBSERVATION showing success: true]

THOUGHT:
Perfect! All evaluations pass now. The itinerary meets all requirements and incorporates the traveler feedback. I can provide the final answer.

ACTION:
{{"tool_name": "final_answer_tool", "arguments": {{"final_output": <final_plan>}}}}

## Critical Requirements (MUST FOLLOW)

1. ✅ ALWAYS run run_evals_tool as your FIRST action
2. ✅ ALWAYS run run_evals_tool again BEFORE calling final_answer_tool  
3. ✅ ONLY call final_answer_tool when run_evals_tool returns success: true
4. ✅ Each response MUST contain BOTH a THOUGHT section AND an ACTION section
5. ✅ Call only ONE tool per ACTION
6. ✅ Use the exact JSON format: {{"tool_name": "...", "arguments": {{...}}}}
7. ✅ Do NOT make up or hallucinate tool results - wait for OBSERVATION

## Context

### TravelPlan Schema (for final output)
{json.dumps(TravelPlan.model_json_schema(), indent=2)}

### Activity Schema
{json.dumps(Activity.model_json_schema(), indent=2)}

### Vacation Information
{json.dumps(vacation_info.model_dump(), indent=2, default=str)}

### Available Weather Data
{json.dumps(weather_for_dates, indent=2)}

### Traveler Feedback (MUST incorporate this)
{TRAVELER_FEEDBACK}

Remember: Think carefully, use tools deliberately, verify your work, and only exit when all evaluations pass!
"""


class ItineraryRevisionAgent(ChatAgent):
    """An agent that revises itineraries using ReAct."""
    system_prompt = ITINERARY_REVISION_AGENT_SYSTEM_PROMPT
    tools = ALL_TOOLS

    def get_observation_string(self, tool_call_obj) -> str:
        """Extracts the observation from the thought-action response."""

        if "tool_name" not in tool_call_obj:
            return "OBSERVATION: No tool name specified."

        if "arguments" not in tool_call_obj:
            return "OBSERVATION: No arguments specified."

        if not isinstance(tool_call_obj["arguments"], dict):
            return f"OBSERVATION: Arguments should be a dictionary, got {type(tool_call_obj['arguments'])} instead."

        if not isinstance(tool_call_obj["tool_name"], str):
            return f"OBSERVATION: Tool name should be a string, got {type(tool_call_obj['tool_name'])} instead."

        tool_name = tool_call_obj["tool_name"]
        arguments = tool_call_obj["arguments"]

        tool_fn = None

        for tool in self.tools:
            if tool.__name__ == tool_name:
                tool_fn = tool
                break

        if tool_fn is None:
            return f"OBSERVATION: Unknown tool name '{tool_name}' in action string."

        try:
            tool_response = tool_fn(**arguments)
            return f"OBSERVATION: Tool {tool_name} called successfully with response: {tool_response}"
        except Exception as e:
            return f"OBSERVATION: Error occurred while calling tool {tool_name}: {e}"

    def run_react_cycle(
        self, original_travel_plan: TravelPlan, max_steps: int = 10, model: Optional[OpenAIModel] = None, client = None,
    ) -> TravelPlan:
        """Runs the ReAct cycle to revise the itinerary based on the evaluation results."""
        from json_repair import repair_json

        self.add_message(
            role="user",
            content=f"Here is the itinerary for revision:\n{original_travel_plan.model_dump_json()}",
        )
        resp = None

        for step in range(max_steps):
            resp = self.get_response(model=model, client=client) or ""

            if "ACTION:" not in resp:
                self.add_message(role="user", content="No action found in response.")
                continue

            action_string = resp.split("ACTION:")[1].strip()

            try:
                action_string = repair_json(action_string)
                tool_call_obj = json.loads(action_string)
            except json.JSONDecodeError:
                print(f"Invalid JSON in action string: {action_string}")
                self.add_message(
                    role="user",
                    content=f"Invalid JSON in action string: {action_string}",
                )
                continue

            tool_name = tool_call_obj.get("tool_name", None)

            if tool_name == "final_answer_tool":
                try:
                    new_travel_plan = TravelPlan.model_validate(
                        tool_call_obj["arguments"].get("final_output", tool_call_obj["arguments"])
                    )
                    return new_travel_plan
                except Exception as e:
                    self.add_message(
                        role="user", content=f"Error validating final answer: {e}"
                    )
                    continue

            else:
                observation_string = self.get_observation_string(
                    tool_call_obj=tool_call_obj
                )
                self.add_message(role="user", content=observation_string)

        raise RuntimeError(
            f"ReAct cycle did not complete within {max_steps} steps. Last response: {resp}"
        )

# Instantiate the Itinerary Revision Agent
itinerary_revision_agent = ItineraryRevisionAgent(client=client, model=MODEL)

# Let's get a single THOUGHT/ACTION response back to check that the agent is working as expected.
resp = itinerary_revision_agent.chat(
    user_message=f"Here is the itinerary for revision: {travel_plan_1.model_dump_json(indent=2)}",
    add_to_messages=False,
    model=MODEL,
    client=client,
) or ""

print_in_box(resp, "Raw Response")
# Check for THOUGHT
if "THOUGHT:" in resp:
    print("✅ `THOUGHT:` found in raw the response, as expected.")
else:
    print("❌ Expected `THOUGHT:` in raw the response. Please check the system prompt (output format).")
# Check for ACTION
if "ACTION:" in resp:
    print("✅ `ACTION:` found in raw the response, as expected.")
else:
    print("❌ Expected `ACTION:` in raw the response. Please check the system prompt (output format).")
if '"tool_name"' in resp:
    print("✅ `\"tool_name\":` found in raw the response, as expected.")
else:
    print("❌ Expected `\"tool_name\":` in raw the response. Please check the system prompt (output format).")

In [ ]:
# Now let's run the ReAct cycle multiple times to get the revised itinerary.
# No changes needed here.

itinerary_revision_agent = ItineraryRevisionAgent(client=client, model=MODEL)
travel_plan_2 = itinerary_revision_agent.run_react_cycle(
    original_travel_plan=travel_plan_1, max_steps=15,
    model=MODEL,
    client=client,
)

print("✅ Revised itinerary generated successfully. Congratulations!")

In [ ]:
# Last let's double check that the revised travel plan passes all evaluation functions.
# No changes needed here.

eval_results_2 = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_2,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

assert eval_results_2.success, f"❌ Read the traces above and modify the system prompt.\n\nFailures: {eval_results_2.failures}"

print("✅ All evaluation functions passed successfully for the revised travel plan.")

eval_results_2

In [ ]:
# Show the final travel plan in a readable format.
# No changes needed here.

from IPython.display import display

for itinerary_day in travel_plan_2.itinerary_days:
    print(f"Date: {itinerary_day.date}")
    print(
        f"Weather: {itinerary_day.weather.condition} ({itinerary_day.weather.temperature}°{itinerary_day.weather.temperature_unit})"
    )

    activities_df = pd.DataFrame(
        [
            activity_recommendation.activity.model_dump()
            for activity_recommendation in itinerary_day.activity_recommendations
        ]
    )
    display(activities_df)

## And, just for fun!

In [ ]:
# And finally, just for fun, let's narrate the trip.
# No changes needed here.

from project_lib import narrate_my_trip

narrate_my_trip(
    vacation_info=vacation_info,
    itinerary=travel_plan_2,
    client=client,
    model=MODEL,
)

## CONGRATULATIONS! 🎉🥳👏

You have successfully planned a stellar vacation to AgentsVille!